# Active Lerning sobre el MNIST

## Imports

### Del proyecto

In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

### Del Framework

In [ ]:
from core import ActiveLearner, Dataset, Oracle
from querys import UncertaintySelector, CertaintySelector, RandomSelector, MinDiffSelector, EntropySelector

## Datos

Para probar el framework lo haré sobre el MNIST. Aprovechando que está todo etiquetado, dejaré un 25% del total para hacer el testing al final de cada iteración. De los datos restantes, tomaré el 5% como datos etiquetados iniciales, y usaré el otro 95% para aplicar el aprendizaje activo.

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target
X, X_test, y, y_test = train_test_split(X, y, test_size=0.33, random_state=3)
X_train, X_unlabeled, y_train, y_unlabeled = train_test_split(X, y, test_size=0.98)

## Instancias mínimas para poder aplicar el framework

Creo una instancia para el **Dataset** y otra para el **Oracle**.

El Oracle lo midifiqué para poder mostrar una imagen del dígito a etiquetar. Luego, para acelerar el proceso de testeo lo volví a modificar para que en lugar de preguntar al usuario etiquetador, directamente devuelva la etiqueta.

In [ ]:
class DigitsOracle(Oracle):
    target_names = list(range(10))
    
    def ask(self, X_readable, recoms):
        return X_readable

    def show_element(self, x, r):
        plt.gray()
        plt.matshow(x[0])
        plt.show()
        print(r)
        print(x[1])

class Digits(Dataset):
    real_y = y_unlabeled
    
    def get_unlabeled_readable(self, i):
        return self.real_y[i]
        #x = self.get_unlabeled()[i]
        #return (x.reshape(8,8), self.real_y[i])

Comparación de la cantidad de datos *iniciales*, para *validación* dentro de cada iteración y para para *testing* final.

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(X_unlabeled))

## Evaluación Final

Comparación de los selectores de Active Learning y también del selector RandomSelector (el cual es equivalente a no aplicar Active Learning. Se compara una vez a cada uno de los selectores del framework excepto el RandomSelector, que es el unico que tiene un comportamiento no determinístico. Por esta razón se ejecutan 5 instancias del mismo y luego se calcula un promedio de ellos para ver el desepempeño medio del mismo.

In [ ]:
selectors = [CertaintySelector, UncertaintySelector, MinDiffSelector, EntropySelector, RandomSelector, RandomSelector, RandomSelector, RandomSelector, RandomSelector]

In [ ]:
selector_scores = []
for selector in selectors:
    dataset = Digits(X_train, y_train, X_unlabeled)
    forest = RandomForestClassifier(random_state=7)
    #forest = LogisticRegression()
    oracle = DigitsOracle()
    al = ActiveLearner(forest, dataset, selector, oracle)
    scores = []
    al.fit()
    scores.append(al.model.score(X_test, y_test))
    for _ in tqdm(range(120)):
        selected = al.select(10)
        y = al.ask(selected)
        al.tag_elements(selected, y)
        al.fit()
        scores.append(al.model.score(X_test, y_test))
    selector_scores.append(scores)

In [ ]:
random_avg = []
for i in range(len(selector_scores[0])):
    suma = 0
    for selector in selector_scores[4:]:
        suma += selector[i]
    random_avg.append(suma/5)

## Visualizaciones

Grafico el desempeño de cada uno de los selectores y el promedio de los RandomSelectors

In [ ]:
import matplotlib
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)

In [ ]:
plt.figure(figsize=(25,15))
plt.title('Comparación de selectores')
plt.xlabel('Iteraciones')
plt.ylabel('Precision')
plt.plot(selector_scores[0])
plt.plot(selector_scores[1])
plt.plot(selector_scores[2])
plt.plot(selector_scores[3])
plt.plot(selector_scores[4], color='gray')
plt.plot(selector_scores[5], color='gray')
plt.plot(selector_scores[6], color='gray')
plt.plot(selector_scores[7], color='gray')
plt.plot(selector_scores[8], color='gray')
#plt.plot(random_avg, color='black')
plt.legend(['Certainty', 'Uncertainty', 'MinDiff', 'Entropy', 'Random'])
plt.savefig('mnist_random_forest')
plt.show()